### Setup database connectivity

We'll reuse our module from the previous notebook (***`00_database_connectivity_setup.ipynb`***) to establish connectivity to the database

In [1]:
%run '00_database_connectivity_setup.ipynb'
IPython.display.clear_output()

### Apache MADlib

Model parallel problems or task parallel problems typically involve building a machine learning model on a dataset that cannot fit into memory, on a distributed cluster. The [Apache MADlib](http://madlib.apache.org/) project explicitly parallelizes such models by splitting them into sub-tasks that can be simultaneously executed on multiple nodes of a cluster and combining the results from these sub-tasks to fit the original model.

MADlib has a very rich collection of machine learning algorithms implemented in-database and is highly performant on large scale datasets.

![Apache MADlib](https://github.com/vatsan/postgresopen-2017/blob/master/docs/images/madlib_1.png?raw=true)

### Breadth of algorithms

![Apache MADlib User Guide](https://github.com/vatsan/postgresopen-2017/blob/master/docs/images/madlib_2.png?raw=true)

### Architecture

![Apache MADlib Architecture](https://github.com/vatsan/postgresopen-2017/blob/master/docs/images/madlib_3.png?raw=true)

### Ridge Regression - Demo

In [2]:
%%time
%%execsql
drop table if exists wine_sample_madlib;
create table wine_sample_madlib
as
(
    select
        id,
        ARRAY[
            fixed_acidity,
            volatile_acidity,
            citric_acid,
            residual_sugar,
            chlorides,
            free_sulfur_dioxide,
            total_sulfur_dioxide,
            density,
            ph,
            sulphates,
            alcohol
        ] as features,
        quality
    from
        wine_sample
)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.8 ms


In [3]:
%%time
%%execsql
drop table if exists wine_sample_madlib_mdl, wine_sample_madlib_mdl_summary;
select 
    madlib.elastic_net_train(
        -- Source table
        'wine_sample_madlib',
        -- Result table
        'wine_sample_madlib_mdl', 
        -- Dependent variable
        'quality',                 
        -- Independent variable
        'features',
        -- Regression family
        'gaussian',             
        -- Alpha value
        0.5,           
        -- Lambda value
        0.1,          
        -- Standardize
        TRUE,
        -- Grouping column(s)
        NULL,         
        -- Optimizer
        'fista',     
        -- Optimizer parameters
        '',   
        -- Excluded columns
        NULL,        
        -- Maximum iterations
        10000,     
        -- Tolerance value
        1e-6                       
    );

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 627 ms


In [4]:
%%time
%%showsql
select
    *
from
    wine_sample_madlib_mdl;

,family,features,features_selected,coef_nonzero,coef_all,intercept,log_likelihood,standardize,iteration_run
0,gaussian,"[features[1], features[2], features[3], features[4], features[5], features[6], features[7], features[8], features[9], features[10], features[11]]","[features[1], features[2], features[5], features[7], features[10], features[11]]","[0.00213245063091, -0.985673014513, -0.273021090308, -0.000964230457507, 0.489861692677, 0.251937832341]","[0.00213245063091, -0.985673014513, 0.0, 0.0, -0.273021090308, 0.0, -0.000964230457507, 0.0, 0.0, 0.489861692677, 0.251937832341]",3.25888,-0.249234,True,64


CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 32.8 ms


### Learn More

More example available here: [Apache MADlib Community Artifacts](https://github.com/apache/madlib-site/tree/asf-site/community-artifacts)